In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy
import seaborn as sns
import plotly.graph_objects as go
import networkx as nx

# OUTLINE

[Datasets](#Datasets)  
- [Algeria export for 2021](#Algeria-2021-Trade-data)  
- [Change column names](#Change-column-names)  
- [Multiple values by 1000](#Change-value-from-thousands-of-dollars-to-dollars)  
- [Country code](#Country-code)   
- [Add importer and exporter names](#Adding-Importer-and-exporter)  
- [Product code](#Product-code)  

[Algeria exports](#Algeria-Exports)  
- [List of countries that receives Algeria Exports](#List-of-countries-that-receives-Algeria-Exports)  
- [Total trade by country](#Total-trade-by-country)  
- [Top 10 countries that imports from Algeria](#Top-10-countries-that-imports-from-Algeria)  
 - [Countries that imports from Algeria with more than billion dollars](#Countries-that-imports-from-Algeria-with-more-than-billion-dollars)  
 - [Countries that imports from Algeria with less than billion dollars](#Countries-that-imports-from-Algeria-with-less-than-billion-dollars)  
 - [Countries that imports from Algeria with less than 100 million dollars](#Countries-that-imports-from-Algeria-with-less-than-100-million-dollars)  
 - [Countries that imports from Algeria with less than 10 million dollars](#Countries-that-imports-from-Algeria-with-less-than-10-million-dollars)  
 - [Countries that imports from Algeria with less than 1 million dollars](#Countries-that-imports-from-Algeria-with-less-than-1-million-dollars)  
 - [Countries that imports from Algeria with less than 100 thousands dollars](#Countries-that-imports-from-Algeria-with-less-than-100-thousands-dollars)  
 - [The number of products imported by each country](#The-number-of-products-imported-by-each-country)
 


# Datasets

## Algeria 2021 Trade data

In [ ]:
df_2021 = pd.read_csv('Algeria_export_transaction.csv')
df_2021

In [ ]:
print('The total number of products code is: {}'.format(len(df_2021)))

In [ ]:
df_2021.columns

**t:**	Year  
**k:**	Product category (HS 6-digit code)  
**i:**	Exporter (ISO 3-digit country code)  
**j:**	Importer (ISO 3-digit country code)  
**v:**	Value of the trade flow (in thousands current USD)  
**q:**	Quantity (in metric tons)  

## Change column names

In [ ]:
df_2021.columns = ['year','exporter_code','importer_code','product_code','value_dollar','quantity_metric_ton']

In [ ]:
df_2021.drop(columns='Unnamed: 0',inplace=True)

In [ ]:
df_2021.columns

In [ ]:
df_2021.columns = ['year','exporter_code','importer_code','product_code','value_dollar','quantity_metric_ton']

In [ ]:
df_2021

## Change value from thousands of dollars to dollars

In [ ]:
df_2021['value_dollar'] = df_2021['value_dollar']*1000
df_2021

## Country code

In [ ]:
df_country = pd.read_csv('country_codes_V202301.csv')
df_country

In [ ]:
df_country['country_code']

In [ ]:
df_country.iloc[0]

In [ ]:
filter_Algeria = df_country['country_name_abbreviation'] == 'Algeria'
filter_Algeria

In [ ]:
Algeria_code = df_country[filter_Algeria]
Algeria_code

In [ ]:
df_country[df_country['country_name_abbreviation'] == 'Algeria']

## Adding Importer and exporter

In [ ]:
importer_list = []
exporter_list = []

for index, row in df_2021.iterrows():
    exporter_code = row['exporter_code']
    importer_code = row['importer_code']
    
    exporter_name = df_country[df_country['country_code']==exporter_code]['country_name_abbreviation']
    importer_name = df_country[df_country['country_code']==importer_code]['country_name_abbreviation']
    '''
    print(list(exporter_name)[0])
    print(list(importer_name)[0])
    break
    '''
    exporter_list.append(list(exporter_name)[0])
    importer_list.append(list(importer_name)[0])
    

In [ ]:
print('length of importer list is: {}'.format(len(importer_list)))
print('length of exporter list is: {}'.format(len(exporter_list)))

In [ ]:
df_2021.insert(1,"exporter_name",exporter_list)
df_2021.insert(3,"importer_name",importer_list)

df_2021

## Product code

**01-05**  Animal & Animal Products  
**06-15**  Vegetable Products  
**16-24**  Foodstuffs  
**25-27**  Mineral Products  
**28-38**  Chemicals & Allied Industries  
**39-40**  Plastics / Rubbers  
**41-43**  Raw Hides, Skins, Leather, & Furs  
**44-49**  Wood & Wood Products  
**50-63**  Textiles  
**64-67**  Footwear / Headgear  
**68-71**  Stone / Glass  
**72-83**  Metals  
**84-85**  Machinery / Electrical  
**86-89**  Transportation  
**90-97**  Miscellaneous  

In [ ]:
df_2021['product_code'] = df_2021['product_code'].apply(lambda x: (str(x).rjust(6, '0')))  

In [ ]:
df_2021

In [ ]:
df_2021.info

# Algeria total export in Dollars

In [ ]:
Alg_exp_total_value = df_2021['value_dollar'].sum()
print('The total of Algeria exports is: ${:,.2f}'.format(Alg_exp_total_value))

## List of countries that receives Algeria Exports

In [ ]:
exp_countries = df_2021['importer_name'].unique()
exp_len       = exp_countries.shape[0]
print('The total number of countris that trade with Algeria is: {}'.format(exp_len))

In [ ]:
print('list of countries:')
print(exp_countries)

## Total trade by country

In [ ]:
exp_countries_grouped = df_2021[['importer_name','value_dollar']].groupby(['importer_name'],as_index=False).sum().sort_values(['value_dollar'],ascending=False).reset_index()
pd.options.display.float_format = '{:,.0f}'.format
exp_countries_grouped.drop(columns=['index'],inplace=True)

exp_countries_grouped

In [ ]:
print('the country that receives the largest  amount of exports is {} with ${:,.2f}'.format(exp_countries_grouped['importer_name'][0],
                                                                                            exp_countries_grouped['value_dollar'][0]))
print('the country that receives the smallest amount of exports is {} with ${:,.2f}'.format(exp_countries_grouped['importer_name'][len(exp_countries_grouped)-1],
                                                                                            exp_countries_grouped['value_dollar'][len(exp_countries_grouped)-1]))


## Top 10 countries that imports from Algeria

In [ ]:
exp_countries_grouped_10 = exp_countries_grouped.loc[0:9,'importer_name':'value_dollar']
exp_countries_grouped_10

In [ ]:
exp_countries_grouped_10['value_dollar'] = round(exp_countries_grouped_10['value_dollar']/1e9,2)
exp_countries_grouped_10

In [ ]:
exp_countries_grouped_10.plot(kind='bar',x='importer_name',y='value_dollar')
'''
modify graph to add the following:
    - y-axis titles to show value in million dollars
    - Title for the graph
    - remove legend
'''

## Countries that imports from Algeria with more than billion dollars

In [ ]:
exp_countries_grouped_billion = exp_countries_grouped[exp_countries_grouped['value_dollar']>=1000000000]
exp_countries_grouped_billion

In [ ]:
exp_countries_grouped_billion.plot(kind='bar',x='importer_name',y='value_dollar')

## Countries that imports from Algeria with less than billion dollars

In [ ]:
exp_countries_grouped_lessBillion = exp_countries_grouped[exp_countries_grouped['value_dollar']<1000000000]
exp_countries_grouped_lessBillion

## Countries that imports from Algeria with less than 100 million dollars

In [ ]:
exp_countries_grouped_100million = exp_countries_grouped[exp_countries_grouped['value_dollar']<100000000]
exp_countries_grouped_100million

## Countries that imports from Algeria with less than 10 million dollars

In [ ]:
exp_countries_grouped_10million = exp_countries_grouped[exp_countries_grouped['value_dollar']<10000000]
exp_countries_grouped_10million

## Countries that imports from Algeria with less than 1 million dollars

In [ ]:
exp_countries_grouped_million = exp_countries_grouped[exp_countries_grouped['value_dollar']<1000000]
exp_countries_grouped_million

## Countries that imports from Algeria with less than 100 thousands dollars

In [ ]:
exp_countries_grouped_100k = exp_countries_grouped[exp_countries_grouped['value_dollar']<100000]
exp_countries_grouped_100k

In [ ]:
less_100k           = len(exp_countries_grouped_100k)
million_100k        = len(exp_countries_grouped_million) - len(exp_countries_grouped_100k)
million_1_10        = len(exp_countries_grouped_10million) - len(exp_countries_grouped_million)  
million_10_100      = len(exp_countries_grouped_100million) - len(exp_countries_grouped_10million)  
million_100_billion = len(exp_countries_grouped_lessBillion) - len(exp_countries_grouped_100million)  
billion             = len(exp_countries_grouped_billion) 

In [ ]:
print('The total number of countries that imports from Algeria is: {}'.format(exp_len))
print('The total number of countries that imports from Algeria with less than 100 thousands dollars is: {}'.format(less_100k))
print('The total number of countries that imports from Algeria with less than 1   million   dollars is: {}'.format(million_100k))
print('The total number of countries that imports from Algeria with 1    to   10  million   dollars is: {}'.format(million_1_10))
print('The total number of countries that imports from Algeria with 10   to   100 million   dollars is: {}'.format(million_10_100))
print('The total number of countries that imports from Algeria with 100  to   1   billion   dollars is: {}'.format(million_100_billion))
print('The total number of countries that imports from Algeria with more than 1   billion   dollars is: {}'.format(len(exp_countries_grouped_billion)))

In [ ]:
Alg_exp_countries_labels = ['less_100k',
                            'million_to_100k',
                            'million_1_to_10',
                            'million_10_to_100',
                            '100million_to_billion',
                            'billion']
Alg_exp_countries_values = [less_100k,
                            million_100k,
                            million_1_10,
                            million_10_100,
                            million_100_billion,
                            billion]

Alg_exp_countries_values_sum = sum(Alg_exp_countries_values)
Alg_exp_countries_percentages = [round((x/Alg_exp_countries_values_sum)*100,2) for x in Alg_exp_countries_values]

In [ ]:
print(Alg_exp_countries_labels)
print(Alg_exp_countries_values)
print(Alg_exp_countries_percentages)

In [ ]:
plt.barh(Alg_exp_countries_labels,Alg_exp_countries_values,label=Alg_exp_countries_labels)
#plt.legend()
plt.title('number of contries importing from Algeria based on imports value')

In [ ]:
# make data
x = [1, 2, 3, 4]
colors = plt.get_cmap('Blues')(np.linspace(0.2, 0.7, len(x)))

# plot
fig, ax = plt.subplots()
ax.pie(Alg_exp_countries_values, radius=3, center=(4, 4),autopct = '%1.1f%%',
       wedgeprops={"linewidth": 1, "edgecolor": "white"}, frame=True)

ax.set(xlim=(0, 8), xticks=np.arange(1, 8),
       ylim=(0, 8), yticks=np.arange(1, 8))
ax.legend(Alg_exp_countries_labels,
          loc ="center left",
          bbox_to_anchor =(1, 0, 0.5, 1))
ax.set_title('percentage of countries according to the value of their imports from Algeria (total 120 countries)')
plt.show()

## The number of products imported by each country

In [ ]:
exp_numberOFproducts_grouped = df_2021[['importer_name','product_code']].groupby(['importer_name'],as_index=False).count().sort_values(by='product_code',ascending=False).reset_index()
pd.options.display.float_format = '{:,.0f}'.format
exp_numberOFproducts_grouped.drop(columns=['index'],inplace=True)

exp_numberOFproducts_grouped

In [ ]:
Alg_exp_exp_sumarry_merged = exp_countries_grouped.merge(exp_numberOFproducts_grouped,how='outer', on='importer_name')
Alg_exp_exp_sumarry_merged

In [ ]:
Alg_exp_goods = copy.copy(df_2021) 
Alg_exp_goods

In [ ]:
Alg_exp_goods['product_code_chapter'] = Alg_exp_goods['product_code'].apply(lambda x: x[0:2] )
Alg_exp_goods['product_code_heading'] = Alg_exp_goods['product_code'].apply(lambda x: x[2:4] )
Alg_exp_goods['product_code_subhead'] = Alg_exp_goods['product_code'].apply(lambda x: x[4:6] )
Alg_exp_goods

In [ ]:
Alg_exp_goods.columns

In [ ]:
Alg_exp_goods = Alg_exp_goods[['year', 
                               'exporter_name', 'exporter_code',
                               'importer_name','importer_code',
                               'product_code',
                               'product_code_chapter',
                               'product_code_heading',
                               'product_code_subhead',
                               'value_dollar',
                               'quantity_metric_ton']]
Alg_exp_goods

In [ ]:
product_chapters = Alg_exp_goods['product_code_chapter'].unique()
print('Total number of product is {} which falls under {} chapters'.format(len(Alg_exp_goods),len(product_chapters)))
product_chapters

## Number of products under each chapter

In [ ]:
Alg_exp_goods_summary = Alg_exp_goods[['exporter_name', 
                                       'importer_name',
                                       'product_code',
                                       'product_code_chapter',
                                       'value_dollar',
                                       'quantity_metric_ton']].groupby(Alg_exp_goods['product_code_chapter']).sum().sort_values(['value_dollar'],ascending=False).reset_index()
#Alg_exp_goods_summary.drop(columns=['index'],inplace=True)
Alg_exp_goods_summary.head(20)

In [ ]:
Alg_exp_goods_summary['exported_value_%'] = Alg_exp_goods_summary['value_dollar'].apply(lambda x: float((x/Alg_exp_total_value)*100))
pd.options.display.float_format = '{:,.00f}'.format
Alg_exp_goods_summary['exported_value_%']

In [ ]:
Alg_exp_goods_summary

In [ ]:
product_chapter_max = Alg_exp_goods_summary.iloc[Alg_exp_goods_summary['value_dollar'].argmax()]
print('The product exported the max   : {}'.format(product_chapter_max['product_code_chapter']))
print('The product exported value     : ${:,.2f}'.format(product_chapter_max['value_dollar']))
print('The product exported percentage: {:.2f}%'.format(product_chapter_max['exported_value_%']))
print('Product code 27 stands for: MINERAL FUELS, MINERAL OILS AND PRODUCTS OF THEIR DISTILLATION; BITUMINOUS SUBSTANCES; MINERAL WAXES')

## Countries that receive Product code 27

In [ ]:
exported_product_27 = Alg_exp_goods[Alg_exp_goods['product_code_chapter']=='27'].sort_values(by=['value_dollar'],ascending=False )
exported_product_27

In [ ]:
exported_product_27_countries = exported_product_27[['importer_name',
                                                    'value_dollar']].groupby('importer_name').sum().sort_values(by=['value_dollar'],ascending=False)
exported_product_27_countries

In [ ]:
exported_product_27_countries_sum = exported_product_27_countries['value_dollar'].sum()
print('total exported of product 27 is: ${:,.2f}'.format(exported_product_27_countries_sum))
exported_product_27_countries['exported_value_%'] = exported_product_27_countries['value_dollar'].apply(lambda x: (x/exported_product_27_countries_sum)*100)
exported_product_27_countries